## Engineered Features For Customer Product Purchase Trends

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_allData = pd.read_pickle('input/all_data.p')

In [3]:
df_allData.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,SucceedingOrdCnt
0,2,33120,1,1,Organic-Egg-Whites,86,16,eggs,dairy eggs,202279,prior,3,5,9,8.0,6
1,2,28985,2,1,Michigan-Organic-Kale,83,4,fresh vegetables,produce,202279,prior,3,5,9,8.0,6
2,2,9327,3,0,Garlic-Powder,104,13,spices seasonings,pantry,202279,prior,3,5,9,8.0,6
3,2,45918,4,1,Coconut-Butter,19,13,oils vinegars,pantry,202279,prior,3,5,9,8.0,6
4,2,30035,5,0,Natural-Sweetener,17,13,baking ingredients,pantry,202279,prior,3,5,9,8.0,6


In [4]:
df_allDataT = df_allData[df_allData['SucceedingOrdCnt']>1]

#### Possibility of ordering a Product

In [5]:
# Count the number of times a product was ordered by a customer.
df_prdCnt = df_allDataT.groupby(['user_id','product_id']).size()

In [6]:
df_prdCnt.name = 'count'

In [7]:
df_prdCnt = df_prdCnt.reset_index()
df_prdCnt.head()

,user_id,product_id,count
0,1,196,9
1,1,10258,8
2,1,10326,1
3,1,12427,9
4,1,13032,2


In [8]:
# Maximum and Minimum number of times a user placed an order to determine possibility of ordering
maxOrd = df_allDataT.groupby('user_id')['order_number'].max().reset_index()

In [9]:
maxOrd.columns = ['user_id','maxOrd']

In [10]:
minPrdOrd = df_allDataT.groupby(['user_id','product_id'])['order_number'].min().reset_index()
minPrdOrd.columns = ['user_id','product_id','minPrdOrd']

In [11]:
poss = minPrdOrd.merge(maxOrd,on='user_id',how='left')

In [12]:
poss['possibility'] = poss['maxOrd']-poss['minPrdOrd']+1

In [13]:
df_comb = df_prdCnt.merge(poss,on=['user_id','product_id'],how='left')

In [14]:
df_comb['ordPoss'] = df_comb['count']/df_comb['possibility']

In [15]:
colm = ['user_id','product_id','possibility','ordPoss']

In [16]:
df_comb.head()

,user_id,product_id,count,minPrdOrd,maxOrd,possibility,ordPoss
0,1,196,9,1,9,9,1.00
1,1,10258,8,2,9,8,1.00
2,1,10326,1,5,9,5,0.20
3,1,12427,9,1,9,9,1.00
4,1,13032,2,2,9,8,0.25


In [17]:
df_comb.to_pickle('input/prdOrdPoss.p')

#### Products Frequency By Customer

In [18]:
# Calculate average, minimum, maximum, median and standard deviation a product is ordered by a user.
orderSze = df_allDataT.groupby('order_id').size().to_frame()

In [19]:
orderSze.columns = ['orderSze']

In [20]:
orderSze.reset_index(inplace=True)

In [21]:
orderSze = orderSze.merge(df_allDataT[['order_id','user_id','product_id']])

In [22]:
df = df_allDataT.sort_values(['user_id','product_id']).drop_duplicates(['user_id','product_id'])[['user_id','product_id']]

In [23]:
df = df.set_index(['user_id','product_id'])

In [24]:
df_grp = orderSze.groupby(['user_id','product_id'])

In [25]:
df_grp.head()

,order_id,orderSze,user_id,product_id
0,2,9,202279,33120
1,2,9,202279,28985
2,2,9,202279,9327
3,2,9,202279,45918
4,2,9,202279,30035
5,2,9,202279,17794
6,2,9,202279,40141
7,2,9,202279,1819
8,2,9,202279,43668
9,3,8,205970,33754


In [26]:
df['userItem_Freq_min'] = df_grp['orderSze'].min()

In [27]:
df['userItem_Freq_max'] = df_grp['orderSze'].max()

In [28]:
df['userItem_Freq_avg'] = df_grp['orderSze'].mean()

In [29]:
df['userItem_Freq_median'] = df_grp['orderSze'].median()

In [30]:
df['userItem_Freq_std'] = df_grp['orderSze'].std()

In [31]:
df.reset_index(inplace=True)

In [32]:
# Calculate order size by user
userOrderSze = orderSze.groupby(['user_id'])['orderSze'].min().to_frame()

In [33]:
userOrderSze.columns = ['userOrderSzeMin']

In [34]:
userOrderSze['userOrderSzeMax']=orderSze.groupby(['user_id'])['orderSze'].max()

In [35]:
userOrderSze.reset_index(inplace=True)

In [36]:
df.head()

,user_id,product_id,userItem_Freq_min,userItem_Freq_max,userItem_Freq_avg,userItem_Freq_median,userItem_Freq_std
0,1,196,4,8,5.555556,5.0,1.130388
1,1,10258,4,8,5.625000,5.5,1.187735
2,1,10326,8,8,8.000000,8.0,NaN
3,1,12427,4,8,5.555556,5.0,1.130388
4,1,13032,5,6,5.500000,5.5,0.707107


In [37]:
df = df.merge(userOrderSze,on='user_id',how='left')

In [38]:
#Calculate co-occurance of order size and item ordered
df['useritem_cooccur-min-min'] = df['userOrderSzeMin']  - df['userItem_Freq_min']
df['useritem_cooccur-max-min'] = df['userItem_Freq_max'] - df['userItem_Freq_min']
df['useritem_cooccur-max-max'] = df['userOrderSzeMax'] - df['userItem_Freq_max']

In [39]:
df.drop(['userOrderSzeMin','userOrderSzeMax'],axis=1,inplace=True)

In [40]:
df.to_pickle('input/ProdFreqCust.p')

#### Total Purchases

In [41]:
orders = pd.read_csv('orders.csv',usecols=['order_id','user_id','order_number'])

In [42]:
label = pd.read_pickle('input/label.p')

In [43]:
label.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,SucceedingOrdCnt,key
0,2550362,196,1,1,Soda,77,7,soft drinks,beverages,1,prior,10,4,8,30.0,1,1 196
1,2550362,10258,6,1,Pistachios,117,19,nuts seeds dried fruit,snacks,1,prior,10,4,8,30.0,1,1 10258
2,2550362,10326,5,0,Organic-Fuji-Apples,24,4,fresh fruits,produce,1,prior,5,4,15,28.0,6,1 10326
3,2550362,12427,9,1,Original-Beef-Jerky,23,19,popcorn jerky,snacks,1,prior,10,4,8,30.0,1,1 12427
4,2550362,13032,8,1,Cinnamon-Toast-Crunch,121,14,cereal,breakfast,1,prior,10,4,8,30.0,1,1 13032


In [44]:
df=label.merge(orders,on=['order_id','user_id'],how='left')

In [45]:
df.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number_x,order_dow,order_hour_of_day,days_since_prior_order,SucceedingOrdCnt,key,order_number_y
0,2550362,196,1,1,Soda,77,7,soft drinks,beverages,1,prior,10,4,8,30.0,1,1 196,10
1,2550362,10258,6,1,Pistachios,117,19,nuts seeds dried fruit,snacks,1,prior,10,4,8,30.0,1,1 10258,10
2,2550362,10326,5,0,Organic-Fuji-Apples,24,4,fresh fruits,produce,1,prior,5,4,15,28.0,6,1 10326,10
3,2550362,12427,9,1,Original-Beef-Jerky,23,19,popcorn jerky,snacks,1,prior,10,4,8,30.0,1,1 12427,10
4,2550362,13032,8,1,Cinnamon-Toast-Crunch,121,14,cereal,breakfast,1,prior,10,4,8,30.0,1,1 13032,10


In [46]:
# Calculate total number of purchases made by a customer who has made more than 1 purchase
total_buy = df_allData[df_allData['SucceedingOrdCnt']>1].groupby(['user_id','product_id']).size().reset_index()

In [47]:
total_buy.columns=['user_id','product_id','total_buy']

In [48]:
df=df.merge(total_buy,on=['user_id','product_id'],how='left')

In [49]:
df.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number_x,order_dow,order_hour_of_day,days_since_prior_order,SucceedingOrdCnt,key,order_number_y,total_buy
0,2550362,196,1,1,Soda,77,7,soft drinks,beverages,1,prior,10,4,8,30.0,1,1 196,10,9
1,2550362,10258,6,1,Pistachios,117,19,nuts seeds dried fruit,snacks,1,prior,10,4,8,30.0,1,1 10258,10,8
2,2550362,10326,5,0,Organic-Fuji-Apples,24,4,fresh fruits,produce,1,prior,5,4,15,28.0,6,1 10326,10,1
3,2550362,12427,9,1,Original-Beef-Jerky,23,19,popcorn jerky,snacks,1,prior,10,4,8,30.0,1,1 12427,10,9
4,2550362,13032,8,1,Cinnamon-Toast-Crunch,121,14,cereal,breakfast,1,prior,10,4,8,30.0,1,1 13032,10,2


In [50]:
df['total_buy_ratio']=df.total_buy/(df.order_number_x-1)

In [51]:
col = ['user_id','order_id','product_id','total_buy','total_buy_ratio']

In [52]:
df[col].to_pickle('input/totalbuys.p')